In [ ]:
import sys
from pathlib import Path

# Add the 'backtest' directory to the system path
notebook_dir = Path().resolve()
backtest_dir = notebook_dir.parent / 'backtest'
sys.path.append(str(backtest_dir))

In [ ]:
import pandas as pd
from backtest import Backtest, Strategy, TradeAction

In [ ]:
# First run the get_data.ipynb notebook to generate the data file
# or copy the code from get_data.ipynb here to download the data directly

# Read the OHLCV data from data/BTCUSDT.csv
data = pd.read_csv('data/BTCUSDT.csv', index_col='Date', parse_dates=True)

In [ ]:
class SmaCrossStrategy(Strategy):

    def __init__(self):
        self.sma_short = 10
        self.sma_long = 40

    def on_candle(self, historical_data, positions_list):
        open_positions = [p for p in positions_list if p.exit_time is None]
        data_with_smas = self.get_data_with_smas(historical_data)
        if self.has_positive_crossing(data_with_smas):
            # Open a long position
            trade_actions = [TradeAction(action="enter", quantity=1)]
            # Close any existing positions
            if len(open_positions) > 0:
                trade_actions += [
                    TradeAction(action="exit", position_id=pos.id)
                    for pos in open_positions
                ]
            return trade_actions
        elif self.has_negative_crossing(data_with_smas):
            # Open a short position
            trade_actions = [TradeAction(action="enter", quantity=-1)]
            # Close any existing positions
            if len(open_positions) > 0:
                trade_actions += [
                    TradeAction(action="exit", position_id=pos.id)
                    for pos in open_positions
                ]
            return trade_actions
        return []
    
    def get_data_with_smas(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        df.loc[:, "sma_short"] = df["Close"].rolling(window=self.sma_short).mean()
        df.loc[:, "sma_long"] = df["Close"].rolling(window=self.sma_long).mean()
        return df

    def has_positive_crossing(self, df: pd.DataFrame) -> bool:
        if len(df) < self.sma_long:
            return False
        last_data = df.iloc[-1]
        prev_data = df.iloc[-2]
        return (last_data["sma_short"] > last_data["sma_long"]) & (
            prev_data["sma_short"] <= prev_data["sma_long"]
        )

    def has_negative_crossing(self, df: pd.DataFrame) -> bool:
        if len(df) < self.sma_long:
            return False
        last_data = df.iloc[-1]
        prev_data = df.iloc[-2]
        return (last_data["sma_short"] < last_data["sma_long"]) & (
            prev_data["sma_short"] >= prev_data["sma_long"]
        )

In [ ]:
backtest = Backtest(data, SmaCrossStrategy)

backtest.run()

backtest.pnl_df

In [ ]:
stats = backtest.stats()

In [ ]:
# Extract long positions (quantity >= 0 or value >= 0)
long_positions = [pos for pos in backtest.positions if pos.quantity >= 0]
# Create a series of long positions entry "Open" prices from backtest.pnl_df
long_entry_series = backtest.pnl_df.loc[[pos.entry_time for pos in long_positions], 'Open']

# Extract short positions (quantity < 0 or value < 0)
short_positions = [pos for pos in backtest.positions if pos.quantity < 0]
# Create a series of short position entry "Open" prices from backtest.pnl_df
short_entry_series = backtest.pnl_df.loc[[pos.entry_time for pos in short_positions], 'Open']

In [ ]:
from matplotlib import pyplot as plt

df = backtest.pnl_df

plt.figure(figsize=(14, 6))
plt.plot(df['Open'], label="Open Price", alpha=0.7)

plt.scatter(long_entry_series.index, long_entry_series, marker='^', color="green", label="Long", s=100)

plt.scatter(short_entry_series.index, short_entry_series, marker='v', color="red", label="Short", s=100)

for idx, price in long_entry_series.items():
    plt.annotate(idx.strftime('%Y-%m-%d'), xy=(idx, price), xytext=(0, 10), 
                 textcoords='offset points', ha='center', fontsize=8, color='green')
    
for idx, price in short_entry_series.items():
    plt.annotate(idx.strftime('%Y-%m-%d'), xy=(idx, price), xytext=(0, -15), 
                 textcoords='offset points', ha='center', fontsize=8, color='red')

plt.title("BTC/USDT with Long/Short Entry Points")
plt.xlabel('Date')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True)
plt.show()

# Plot the Total PnL
plt.figure(figsize=(14, 2))
plt.plot(df['total_pnl'], label="Total PnL", alpha=0.7)
plt.title("Total PnL Over Time")
plt.xlabel('Date')
plt.ylabel('Total PnL (USDT)')
plt.legend()
plt.grid(True)
plt.show()